# Runs the Heat Pump Model for Multiple Scenaries across Multiple Communities

In [11]:
import sys
import numpy as np
import pandas as pd
from itertools import product

## Loop through Cities Running all Scenarios

In [2]:
# Import heat pump calculation model and library
import heatpump.library as lib
import heatpump.hp_model as hpmod

acquiring library data...


In [3]:
# Base Case Inputs to the Heat Pump Calcuator.
in_base = dict(
    bldg_name='Test',
    notes='',
    pce_limit=500,
    co2_lbs_per_kwh=1.5,
    exist_fuel_use=None,
    elec_uses='all',
    exist_heat_effic=0.8,
    exist_kwh_per_mmbtu=6.25,
    includes_dhw=False,
    includes_dryer=False,
    includes_cooking=False,
    occupant_count=3,
    hp_model_id=-1,
    low_temp_cutoff=5,
    off_months_chks=[],
    garage_stall_count=1,
    garage_heated_by_hp=False,
    indoor_heat_setpoint=70,
    insul_level=2,
    pct_exposed_to_hp=0.46,
    doors_open_to_adjacent=True,
    bedroom_temp_tolerance='med',
    rebate_dol=0,
    pct_financed=0,
    loan_term=10,
    loan_interest=0.05,
    hp_life=14,
    op_cost_chg=0,
    discount_rate=0.05,
    inflation_rate=0.02,
    fuel_esc_rate=0.03,
    elec_esc_rate=0.02,# also chagne it!
    temp_projection=0
)

In [4]:
in_test = in_base.copy()
in_test['city_id'] = 1
in_test['utility'] = lib.util_from_id(53)
in_test['bldg_floor_area'] = 1700
in_test['exist_heat_fuel_id'] = 4     # 4 - #1 Oil, 3 - Propane, 1 - Electricity
in_test['exist_unit_fuel_cost'] = 2.69
in_test['elec_use_jan'] = 674
in_test['elec_use_may'] = 503
in_test['capital_cost'] = 4000
in_test['sales_tax'] =  0.06

In [5]:
in_test['temp_projection'] = 3

In [6]:
mod = hpmod.HP_model(**in_test)
mod.run()
mod.summary

{'fuel_unit': 'gallon',
 'fuel_desc': '#1 Oil',
 'design_heat_load': 27746.8254,
 'design_heat_temp': -4.5,
 'cop': 2.5152836311025726,
 'hp_max_capacity_5F': 11978.4231,
 'max_hp_reached': True,
 'co2_lbs_saved': 1593.2602959990054,
 'co2_driving_miles_saved': 1786.428607787697,
 'hp_load_frac': 0.7807801416606815,
 'avg_temp_monthly': [20.623709677419352,
  20.90160714285714,
  28.822661290322582,
  40.8675,
  52.23596774193548,
  59.612,
  61.77209677419355,
  59.69145161290323,
  54.30225,
  37.020161290322584,
  25.06225,
  21.776532258064517],
 'freezing_days_prop': 0.3452054794520548,
 'heating_days_above5': 351,
 'heating_days_below5': 11,
 'irr': -0.37886037653938354,
 'npv': -4789.364174955661,
 'fuel_use_base': 573.8184854634125,
 'fuel_use_hp': 127.87712208922332,
 'fuel_use_chg': -445.9413633741891,
 'fuel_price_incremental': 2.8514000000000004,
 'elec_use_base': 6567.6264532423575,
 'elec_use_hp': 12096.76582692536,
 'elec_use_chg': 5529.139373683003,
 'elec_rate_avg_base

In [7]:
# Maps Census Area to average size of home.  Comes from the 2018 AHFC
# Housing Assessment.
census_to_size = {
 'Anchorage municipality': 2057,
 'Aleutians West Census Area': 1130,
 'Aleutians East Borough': 1296,
 'Yukon-Koyukuk Census Area': 947,
 'Hoonah-Angoon Census Area': 1258,
 'Bethel Census Area': 950,
 'Juneau City and Borough': 1725,
 'North Slope Borough': 1164,
 'Southeast Fairbanks Census Area': 1745,
 'Matanuska-Susitna Borough': 1912,
 'Denali Borough': 1736,
 'Lake and Peninsula Borough': 1026,
 'Valdez-Cordova Census Area': 1781,
 'Prince of Wales-Hyder Census Area': 1246,
 'Kenai Peninsula Borough': 1890,
 'Northwest Arctic Borough': 920,
 'Nome Census Area': 1168,
 'Wade Hampton Census Area': 834,
 'Dillingham Census Area': 1150,
 'Bristol Bay Borough': 1445,
 'Haines Borough': 1730,
 'Yakutat City and Borough': 1116,
 'Skagway Municipality': 1672,
 'Sitka City and Borough': 1629,
 'Wrangell City and Borough': 1766,
 'Petersburg Borough': 1714,
 'Ketchikan Gateway Borough': 1734,
 'Kodiak Island Borough': 1790,
 'Fairbanks North Star Borough': 2068,
}

In [8]:
# Fuel Prices to override what is in the library.  Key is
# (City Name, Fuel Type ID).
"""
fuel_override = {
    ('Ambler', 4): 9.53/1.03,
    ('Buckland', 4): 6.89/1.06,
    ('Deering', 4): 4.90/1.03,
    ('Kiana', 4): 5.67/1.03,
    ('Kivalina', 4): 4.49/1.03,
    ('Kobuk', 4): 8.24/1.03,
    ('Kotzebue', 4): 6.33/1.06,
    ('Noatak', 4): 10.29/1.03,
    ('Noorvik', 4): 5.64/1.04,
    ('Selawk', 4): 7.99/1.06,
    ('Shungnak', 4): 8.42/1.03,
}
"""
fuel_override = {}

In [9]:
# Some Electric Rate & PCE Overrides
# Key is the Utility ID
"""
block1_rate_override = {
    'Ipnatchiaq Electrric Co - Residential': 0.6547,              # Deering Residential
    'Ipnatchiaq Electrric Co - Commercial - Sm': 0.6547,             # Deering Small Commercial
}

pce_override = {
    'Ipnatchiaq Electrric Co - Residential': 0.2635,              # Deering Residential
    'Ipnatchiaq Electrric Co - Commercial - Sm': 0.2635,             # Deering Small Commercial
}
"""
block1_rate_override = {}
pce_override = {}
def check_elec_override(util):
    """'util' is a Utility series and it is adjusted if one of the
    above overrides applies.
    """
    ut = util.copy(deep=True)
    if ut['Name'] in block1_rate_override:
        kwh_limit, rate = ut.Blocks[0]
        blocks = ut.Blocks.copy()
        blocks[0] = (kwh_limit, block1_rate_override[ut['Name']])
        ut.Blocks = blocks
        
    if ut['Name'] in pce_override:
        ut.PCE = pce_override[ut['Name']]
        
    return ut        

In [10]:
def find_res_util(city):
    """Returns the first residential electric utility for the city, or None
    if no Residential utility exists.
    """
    for _, util_id in city.ElecUtilities:
        util = lib.util_from_id(util_id)
        if not util.IsCommercial:
            return check_elec_override(util)
    return None

def find_sm_comm_util(city):
    """Returns the small commercial utilty for the city, resorting to the 
    only utility if there are no commercial utilities.
    """
    comm_utils = []    # list of commercial utilities
    for _, util_id in city.ElecUtilities:
        util = lib.util_from_id(util_id)
        if util.IsCommercial:
            comm_utils.append(util)
            if np.nan_to_num(util.DemandCharge)==0.0 or 'sm' in util.Name.lower():
                return check_elec_override(util)
    if len(comm_utils) > 0:
        # Assume first commercial utility is the small commercial rate schedule 
        return check_elec_override(comm_utils[0])
    else:
        # Return the first utility
        _, util_id = city.ElecUtilities[0]
        return check_elec_override(lib.util_from_id(util_id))
    return None
    
def scenarios(city_id, fuel_id):
    '''Returns a list of (scenario name, input dictionary) tuples for all 
    of the scenarios being modeled for 'city_id' with the existing system
    using 'fuel_id'.
    '''
    scenarios = []
    city = lib.city_from_id(city_id)
    
    # ------------------- Base Case Scenario
    
    sc_base = in_base.copy()
    sc_base['city_id'] = city_id
    sc_base['exist_heat_fuel_id'] = fuel_id
    
    # If electric heat, then 100% efficient
    if fuel_id == 1:
        sc_base['exist_heat_effic'] = 1.0

    # Capital Cost
    # Each cost level is the same percentage above the one prior.
    # Assume highest level (level 5) is 1.6 x lowest level.
    cost_mult = 1.6 ** 0.25 
    sc_base['capital_cost'] = 4000.0 * cost_mult ** (city.ImprovementCostLevel - 1)
    
    # Find the residential utility for this city
    sc_base['utility'] = find_res_util(city)
    
    # Building Floor Area
    sc_base['bldg_floor_area'] = census_to_size[city.census_area]
    
    # fuel price
    if fuel_id != 1:
        # Not Electricity
        if (city.Name, fuel_id) in fuel_override:
            # Override exists for fuel price
            sc_base['exist_unit_fuel_cost'] =  fuel_override[(city.Name, fuel_id)] 
        else:
            the_fuel = lib.fuel_from_id(fuel_id)
            price_col = the_fuel['price_col']
            price = np.nan_to_num(city[price_col])
            sc_base['exist_unit_fuel_cost'] = price 
    else:
        sc_base['exist_unit_fuel_cost'] = np.nan
        
    # January and May electricity use
    sc_base['elec_use_jan'] = city.avg_elec_usage[0] 
    sc_base['elec_use_may'] = city.avg_elec_usage[4] 
    
    # Sales Tax
    sc_base['sales_tax'] = np.nan_to_num(city.MunicipalSalesTax) + np.nan_to_num(city.BoroughSalesTax)
    
    scenarios.append(('Base', sc_base))
    
    # # # --------------------- Small Load Scenario
    
    # sc_small = sc_base.copy()
    # # 40% smaller floor area
    # sc_small['bldg_floor_area'] *= 0.6 
    
    # # 20% reduction in base electric use
    # sc_small['elec_use_jan'] *= 0.8 
    # sc_small['elec_use_may'] *= 0.8
    
    # # Toyostove or Rinnai type efficiency and auxiliary use
    # if fuel_id == 4:
    #     sc_small['exist_heat_effic'] = 0.84
    # elif fuel_id == 3 or fuel_id==2:
    #     sc_small['exist_heat_effic'] = 0.82
    # sc_small['exist_kwh_per_mmbtu'] = 3.0
    
    # scenarios.append(('Small Load', sc_small))
    
    # # --------------------- Large Load Scenario

    # sc_large = sc_base.copy()
    # # 50% larger floor area
    # sc_large['bldg_floor_area'] *= 1.5      
    
    # # Trying to make this a heat pump favorable scenario, so leave base
    # # electric use the same.
    
    # # Lower non-electric efficiencies, higher auxiliary use
    # if fuel_id != 1:
    #     sc_large['exist_heat_effic'] = 0.77
    #     sc_large['exist_kwh_per_mmbtu'] = 9.0
    
    # scenarios.append(('Large Load', sc_large))
    
    # # -------------------- Base Case with Low Base Electricity Use
    # # This only affects PCE communities, so only do it there.
    # util = sc_base['utility']
    # if (util is not None) and np.nan_to_num(util.PCE) > 0.0:
    
    #     sc_low_elec = sc_base.copy()

    #     # 30% reduction in base electric use
    #     sc_low_elec['elec_use_jan'] *= 0.7
    #     sc_low_elec['elec_use_may'] *= 0.7

    #     scenarios.append(('Low Elec', sc_low_elec))
        
    #     # ----------------- Base Case with No PCE
    #     # Also done only in PCE communities
    #     sc_no_pce = sc_base.copy()
    #     sc_no_pce['pce_limit'] = 0.0
    #     scenarios.append(('No PCE', sc_no_pce))

    # # ---------------------- Community Building, all kWh get PCE.
    # # Only true for communities that have not used their PCE allotment.
    # # Only run this scenario for communities with PCE.
    
    # # Get small commercial electric utility
    # util = find_sm_comm_util(city)
    # if (util is not None) and np.nan_to_num(util.PCE) > 0.0:
        
    #     sc_commun = sc_base.copy()
    #     sc_commun['utility'] =  util
        
    #     # 2,000 square foot building
    #     sc_commun['bldg_floor_area'] = 2000.0

    #     # Base electric use doesn't matter.

    #     # Set infinite PCE limit. 
    #     sc_commun['pce_limit'] = np.inf
        
    #     scenarios.append(('Community Bldg', sc_commun))
    
    return scenarios

In [12]:
# set up all unique projections
temp_projection = [0, 1.5, 3]
rebate_dol = [0, 2000, 4000]
fuel_esc_rate = [0.03, 0.06, 0.09]

all_projections = list(product(temp_projection, rebate_dol, fuel_esc_rate))

In [16]:
# simulate estimates for all scenarios 
sc_count = 0
recs = []

for city_name, city_id in lib.cities():
    print(city_name)
    city = lib.city_from_id(city_id)
    
    for fuel_id in (1, 2, 3, 4):
        for scenario_name, inp in scenarios(city_id, fuel_id):

            # If no utility for the scenario or no fuel price if non-electric, skip it
            if (inp['utility'] is None) or (fuel_id != 1 and inp['exist_unit_fuel_cost']==0.0):
                continue
            
            for temp_projection, rebate_dol, fuel_esc_rate in all_projections:
                inp['temp_projection'] = temp_projection
                inp['rebate_dol'] = rebate_dol
                inp['fuel_esc_rate'] = fuel_esc_rate
                inp['elec_esc_rate'] = fuel_esc_rate

                rec = {
                    'City': city_name,
                    'Exist_Fuel_Type': fuel_id,
                    'Scenario': scenario_name,
                    'Rebate_dol': inp['rebate_dol'],
                    'Fuel_Esc_Rate': inp['fuel_esc_rate'],
                    'Temp_Projection': inp['temp_projection'],
                    'Census_Area': city.census_area,
                    'ANCSA_Region': city.ancsa_region,
                    'Util_Name': inp['utility'].Name,
                    'PCE': np.nan_to_num(inp['utility'].PCE),
                    'Sq_Ft': inp['bldg_floor_area'],
                    'Capital_Cost': inp['capital_cost'],
                    'Elec_Use_Jan': inp['elec_use_jan'],
                    'Elec_Use_May': inp['elec_use_may'],
                    'Exist_Unit_Fuel_Cost': inp['exist_unit_fuel_cost']
                }

                md = hpmod.HP_model(**inp)
                md.run()
                smy = md.summary
                rec.update(
                    Design_Heat_Load = smy['design_heat_load'],
                    Design_Heat_Temp = smy['design_heat_temp'],
                    COP = smy['cop'],
                    Max_HP_Cap_Reached = smy['max_hp_reached'],
                    HP_Load_Frac = smy['hp_load_frac'],
                    Avg_Temp_Monthly = smy['avg_temp_monthly'],
                    Freezing_Days_Prop = smy['freezing_days_prop'],
                    Heating_Days_Above5 = smy['heating_days_above5'],
                    Heating_Days_Below5 = smy['heating_days_below5'],
                    IRR = smy['irr'],
                    NPV = smy['npv'],
                    CO2_lbs_saved = smy['co2_lbs_saved'],
                    CO2_driving_miles_saved = smy['co2_driving_miles_saved'],
                    Fuel_Use_Chg = smy['fuel_use_chg'],
                    Fuel_Price_Incremental = smy['fuel_price_incremental'],
                    Elec_Use_Chg = smy['elec_use_chg'],
                    Elec_Rate_Incremental = smy['elec_rate_incremental'],          
                    Elec_Rate_Avg_Base = smy['elec_rate_avg_base']  
                )
                recs.append(rec)
                sc_count += 1
    #if city_name == 'Adak':
    #    break
print(sc_count)
df_results = pd.DataFrame(recs)

# Add categories of cost-effectiveness
df_results['Econ'] = 'Close'
df_results.loc[df_results.NPV < -2000, 'Econ'] =  'Not Economic'
df_results.loc[df_results.NPV > 0, 'Econ'] =  'Economic'

# Add Fuel Type Label
fuel_id_to_label = dict(zip(lib.df_fuel.index, lib.df_fuel.desc))
df_results['Exist_Fuel'] = df_results.Exist_Fuel_Type.map(fuel_id_to_label)

# Save results
df_results.to_csv('results_BoroughProjs.csv')

Adak
Akhiok
Akiachak
Akiak
Akutan
Alakanuk
Alatna
Aleknagik
Allakaket
Ambler
Anaktuvuk Pass - C
Anaktuvuk Pass Res
Anchor Point
Anchorage
Anderson
Angoon
Aniak
Anvik
Arctic Village
Atka
Atmautluak
Atqasuk - Com
Atqasuk - Res
Auke Bay
Barrow/Utqiavik
Beaver
Bethel
Bettles
Big Lake
Birch Creek
Bird Creek
Brevig Mission
Buckland
Cantwell
Central
Chalkyitsik
Chefornak
Chenega
Chevak
Chickaloon
Chicken
Chignik
Chignik Lagoon
Chignik Lake
Chiniak
Chistochina
Chitina
Chuathbaluk
Chugiak
Circle
Clam Gulch
Clark's Point
Clear
Coffman Cove
Cold Bay
Cooper Landing
Copper Center
Cordova
Craig
Crooked Creek
Deering
Delta Junction
Denali Nat'l Park
Dillingham
Diomede
Dot Lake
Douglas
Dutch Harbor
Eagle
Eagle River
Eek
Egegik
Ekwok
Elfin Cove
Elim
Emmonak
English Bay/Nanwal
Ester
Evansville
Eyak
Fairbanks
False Pass
Faribanks - Chena Ridge
Fort Yukon
Gakona
Galena
Gambell
Girdwood
Glennallen
Golovin
Goodnews Bay
Grayling
Gulkana
Gustavus
Haines
Healy
Hollis
Holy Cross
Homer
Hoonah
Hooper Bay
Hope
Hou